<a href="https://colab.research.google.com/github/22842219/ET-DeepSDD/blob/main/How_to_play_our_model_on_Google_Colab%3F_by_ZZY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## How to run our experiments?
Firstly, git clone our repository to your google drive and choose runtime "GPU".  

Secondly, open file "config.json" to config the experiment.  

Then, run our experiments as the following tutorial.

In [86]:
import os
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/ET-DeepSDD"
os.chdir(path)
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['nfgec_evaluate.py',
 'progress_bar.py',
 'logger.py',
 'README.md',
 'load_config.py',
 'train.py',
 'evaluate.py',
 'data',
 '__pycache__',
 'bert',
 '.git',
 'runs',
 'models',
 'cased_L-12_H-768_A-12',
 'Reasoning_process_explanation_by_ZZY.ipynb',
 'build_data.py',
 'data_utils.py',
 'config.json',
 'model.py',
 'cased_L-12_H-768_A-12.zip',
 'out.file',
 'cased_L-12_H-768_A-12.zip.1']

In [2]:
from pathlib import Path
here = Path(path)
print(here)


/content/drive/MyDrive/ET-DeepSDD


In [ ]:
import sys, os, re
import json
import numpy as np
import math


"""
Creates the data distribution of each sample rate from the raw whole dataset. 
In order to have the fair enough comparision and the test if our model is sensitive with the volume of dataset or not,
we use the same validation and test datsets.
The sample rate considered: [0.1:0.9].
"""

sample_ratio = [0.2, 0.5, 0.6, 0.7, 0.8, 0.9]
infile = r"data/datasets/bbn_modified"
# infile = '{}/{}'.format(basefile, dataset)
for file in os.listdir(infile):
  filepath = '{}/{}'.format(infile,file)
  with open(filepath) as f:
    records = [json.loads(x) for x in f]

  count = len(records)
  print("count:", count)

  for r in sample_ratio:
    folder = '_{}/'.format(r)
    folderpath = infile + folder
    # print("filepath:", filepath)

    if not os.path.exists(os.path.dirname(folderpath)):
      try:
        os.makedirs(os.path.dirname(folderpath))
      except OSError as exc:
        if exc.errno != errno.EEXITST:
          raise

    if file == "train.json":
      sample_count = math.ceil(count * r)
    else:
      sample_count = count
    sample_indexes = np.random.choice(
      count,
      sample_count)

    sample_records = []
    for sample_index in sample_indexes:
      sample_record = records[sample_index]
      sample_records.append(sample_record)

    assert len(sample_records) == sample_count
    print(folderpath)

    with open(here / folderpath/file,"a") as f:
      for record in sample_records:
        f.write(json.dumps(record) + "\n")
    print("Sampled {} records from {} original modified records.".format( sample_count, count))



In [3]:
!pip install colorama
!pip install jsonlines
!pip install pysdd
!pip install PySDD


     |████████████████████████████████| 3.5MB 6.8MB/s 
  Created wheel for pysdd: filename=PySDD-0.2.10-cp37-cp37m-linux_x86_64.whl size=2321861 sha256=c60fa9332fe6758509a6cc05b500ef55a8cc0d3b65231c3be33525d7a8ca6caa
  Stored in directory: /root/.cache/pip/wheels/2e/b6/9a/1c7f59c5dee1eb2bd9d606d26b39ee8b9473ceb568fc91d54d
Successfully built pysdd


## Download and activate bert

In [4]:
!pip install bert-serving-client
!pip install -U bert-serving-server[http]

     |████████████████████████████████| 71kB 3.9MB/s 
     |████████████████████████████████| 358kB 7.2MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=f065030d7493987eb5f55bd97a899e3840c24f3962faa9b03dfa49f0fea24885
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil


In [5]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip

--2021-03-25 04:25:45--  https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.196.128, 64.233.191.128, 142.250.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.196.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 404261442 (386M) [application/zip]
Saving to: ‘cased_L-12_H-768_A-12.zip.1’

cased_L-12_H-768_A- 100%[===================>] 385.53M  61.1MB/s    in 6.5s    

2021-03-25 04:25:52 (59.8 MB/s) - ‘cased_L-12_H-768_A-12.zip.1’ saved [404261442/404261442]



In [6]:
!unzip cased_L-12_H-768_A-12

Archive:  cased_L-12_H-768_A-12.zip
replace cased_L-12_H-768_A-12/bert_model.ckpt.meta? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: cased_L-12_H-768_A-12/bert_model.ckpt.meta  
replace cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  y
y

replace cased_L-12_H-768_A-12/vocab.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: cased_L-12_H-768_A-12/vocab.txt  
replace cased_L-12_H-768_A-12/bert_model.ckpt.index? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: cased_L-12_H-768_A-12/bert_model.ckpt.index  
replace cased_L-12_H-768_A-12/bert_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: cased_L-12_H-768_A-12/bert_config.json  


In [7]:
!nohup bert-serving-start -model_dir=./cased_L-12_H-768_A-12 > out.file 2>&1 &max_seq_len=10

## Get stated to run the model

In [8]:
!python build_data.py


INFO  Created new log directory models/bbn_modified_0.2_baseline_March25_sdd_[use_hierarchy = True, use_context_encoders = False, context_window = 10, mention_window = 10, attention_type = none, use_bilstm = False, use_marginal_ranking_loss = False]/bbn_modified_0.2_[all train, all test]/debug/log_25-03-2021_04:29:29.
DEBUG Dumped config to models/bbn_modified_0.2_baseline_March25_sdd_[use_hierarchy = True, use_context_encoders = False, context_window = 10, mention_window = 10, attention_type = none, use_bilstm = False, use_marginal_ranking_loss = False]/bbn_modified_0.2_[all train, all test]/debug/log_25-03-2021_04:29:29/config.txt.
DEBUG Dumped config to models/bbn_modified_0.2_baseline_March25_sdd_[use_hierarchy = True, use_context_encoders = False, context_window = 10, mention_window = 10, attention_type = none, use_bilstm = False, use_marginal_ranking_loss = False]/bbn_modified_0.2_[all train, all test]/debug/config.txt.
DEBUG CONFIGDIR=%s
DEBUG (private) matplotlib data path: %s


In [9]:
import sys
import time
from colorama import Fore, Back, Style

In [ ]:
!pip install cloud-tpu-client

In [ ]:
!python3 train.py

INFO  Created new log directory models/bbn_modified_0.2_baseline_March25_sdd_[use_hierarchy = True, use_context_encoders = False, context_window = 10, mention_window = 10, attention_type = none, use_bilstm = False, use_marginal_ranking_loss = False]/bbn_modified_0.2_[all train, all test]/debug/log_25-03-2021_04:30:30.
DEBUG Dumped config to models/bbn_modified_0.2_baseline_March25_sdd_[use_hierarchy = True, use_context_encoders = False, context_window = 10, mention_window = 10, attention_type = none, use_bilstm = False, use_marginal_ranking_loss = False]/bbn_modified_0.2_[all train, all test]/debug/log_25-03-2021_04:30:30/config.txt.
DEBUG Dumped config to models/bbn_modified_0.2_baseline_March25_sdd_[use_hierarchy = True, use_context_encoders = False, context_window = 10, mention_window = 10, attention_type = none, use_bilstm = False, use_marginal_ranking_loss = False]/bbn_modified_0.2_[all train, all test]/debug/config.txt.
DEBUG CONFIGDIR=%s
DEBUG (private) matplotlib data path: %s
